In [260]:
import pandas as pd
import numpy as np

In [46]:
card_df = pd.read_csv("./월별데이터 정리/month_card_spending.csv")

In [47]:
card_df['m'] = pd.to_datetime(card_df['m']) #datetime type
card_df['median'] = card_df["sum_use_amt"]/card_df["sum_use_cnt"] # 평균계산

In [48]:
card_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20005 entries, 0 to 20004
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   m            20005 non-null  datetime64[ns]
 1   emd_cd       20005 non-null  object        
 2   emd_nm       20005 non-null  object        
 3   count_num    20005 non-null  int64         
 4   mct_cat_cd   20005 non-null  object        
 5   mct_cat_nm   20005 non-null  object        
 6   sum_use_cnt  20005 non-null  float64       
 7   sum_use_amt  20005 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 1.2+ MB


In [52]:
# 중복되는 정보인 emd_cd, mct_cat_cd 삭제
# count_num : 각 달에 소비내역이 존재하는 날인데,,, 뭔가 이거까지 넣기에는 변수가 너무 많아질 것 같아서 일단 뺐어요.
card_df = card_df.drop(columns=['emd_cd','mct_cat_cd','count_num'])

In [259]:
card_df["sum_use_amt"].to_matrix

AttributeError: 'Series' object has no attribute 'to_matrix'

In [280]:
## 새로운 테이블 생성

# 업종명을 담은 리스트
cat_list = list(card_df['mct_cat_nm'].unique())

# 각 동의 이름을 담은 리스트 생성
region_list = list(card_df['emd_nm'].unique())


# 2018-01-01 ~ 2021-06-01 (주기: 월) 날짜에 동별 정보를 담은 dataframe
dt_index = pd.date_range(start='20180101', end='20210601',freq='MS')
val1 = list(dt_index)*len(region_list)
val2 = []
for i in region_list:
    for j in range(0,len(dt_index)):
        val2.append(i)
        
date_df = pd.DataFrame({'Year_Month':val1, 'emd_nm':val2})


In [255]:
val1

[Timestamp('2018-01-01 00:00:00', freq='MS'),
 Timestamp('2018-02-01 00:00:00', freq='MS'),
 Timestamp('2018-03-01 00:00:00', freq='MS'),
 Timestamp('2018-04-01 00:00:00', freq='MS'),
 Timestamp('2018-05-01 00:00:00', freq='MS'),
 Timestamp('2018-06-01 00:00:00', freq='MS'),
 Timestamp('2018-07-01 00:00:00', freq='MS'),
 Timestamp('2018-08-01 00:00:00', freq='MS'),
 Timestamp('2018-09-01 00:00:00', freq='MS'),
 Timestamp('2018-10-01 00:00:00', freq='MS'),
 Timestamp('2018-11-01 00:00:00', freq='MS'),
 Timestamp('2018-12-01 00:00:00', freq='MS'),
 Timestamp('2019-01-01 00:00:00', freq='MS'),
 Timestamp('2019-02-01 00:00:00', freq='MS'),
 Timestamp('2019-03-01 00:00:00', freq='MS'),
 Timestamp('2019-04-01 00:00:00', freq='MS'),
 Timestamp('2019-05-01 00:00:00', freq='MS'),
 Timestamp('2019-06-01 00:00:00', freq='MS'),
 Timestamp('2019-07-01 00:00:00', freq='MS'),
 Timestamp('2019-08-01 00:00:00', freq='MS'),
 Timestamp('2019-09-01 00:00:00', freq='MS'),
 Timestamp('2019-10-01 00:00:00', 

In [316]:
# 값 채우기
# 되긴 되는데 너무 오래걸린다...

median_list = list(np.array(card_df["median"].tolist())) 
cat_array = np.zeros(shape=(len(date_df),len(cat_list))) #조건에 맞는 평균값을 저장할 배열


for i in range(0,len(date_df)):
    date = date_df["Year_Month"][i]
    emd = date_df['emd_nm'][i]
    for j in range(0,len(cat_list)):
        sample_list = card_df[(card_df['m']==date)
                      &(card_df['emd_nm'] == emd)
                      &(card_df['mct_cat_nm']==cat_list[j])].index.tolist()
        if sample_list == []:
            cat_array[i,j] = np.NaN
        else:
            cat_array[i,j] = median_list[sample_list[0]]

In [320]:
# 위에서 만든 median 값이 담긴 array 배열을 데이터 프레임에 저장
cat_df = pd.DataFrame(cat_array, columns = cat_list)

In [324]:
# date_df, cat_df 병합
card_df_new = pd.concat([date_df, cat_df], axis=1)

In [332]:
card_df_new

,Year_Month,emd_nm,간식,농축수산물,마트/슈퍼마켓,배달,부페,식품,아시아음식,양식,주점및주류판매,패스트푸드,한식
0,2018-01-01,건입동,11611.036784,106259.712184,34960.851143,19780.919658,92397.272727,49201.660617,35641.942886,33591.553037,32659.812641,16067.324198,34176.972390
1,2018-02-01,건입동,13200.743771,129250.812062,41764.770832,18826.683521,116451.857143,60057.002143,35777.342187,41546.210117,46362.735849,16735.976110,34383.058920
2,2018-03-01,건입동,11118.287795,80525.058030,32996.785018,19447.298319,107885.760000,52634.300075,33289.151541,44594.545626,32807.068966,15504.352393,32613.028016
3,2018-04-01,건입동,11928.152409,103666.390994,32784.623009,18544.462774,56713.466667,50423.624625,38375.738513,46064.750219,38741.415385,15263.421472,33976.176171
4,2018-05-01,건입동,12378.112014,74175.581937,33580.338901,20405.593880,84271.500000,49263.059602,36232.578479,35229.278632,38258.810526,16030.831987,33646.099706
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1843,2021-02-01,효돈동,13026.598726,56997.741585,52770.552979,22097.697226,506000.000000,34924.625585,32787.755162,35352.533333,47377.578947,15485.766559,33283.108802
1844,2021-03-01,효돈동,11439.275330,50868.001608,50008.374696,19473.936575,NaN,23937.313054,35503.296137,34423.222222,41898.412121,17183.128071,31484.521323
1845,2021-04-01,효돈동,11110.310428,82127.390347,46488.787724,20422.919469,NaN,36128.328099,32163.787179,41016.687179,39490.690909,16018.479027,32371.645395
1846,2021-05-01,효돈동,12239.600914,72589.104651,47158.580090,20983.949077,NaN,24712.667210,37395.237209,42133.709091,45912.662069,18672.407010,32473.150359


In [333]:
card_df_new[card_df_new["emd_nm"]=="건입동"]

,Year_Month,emd_nm,간식,농축수산물,마트/슈퍼마켓,배달,부페,식품,아시아음식,양식,주점및주류판매,패스트푸드,한식
0,2018-01-01,건입동,11611.036784,106259.712184,34960.851143,19780.919658,92397.272727,49201.660617,35641.942886,33591.553037,32659.812641,16067.324198,34176.972390
1,2018-02-01,건입동,13200.743771,129250.812062,41764.770832,18826.683521,116451.857143,60057.002143,35777.342187,41546.210117,46362.735849,16735.976110,34383.058920
2,2018-03-01,건입동,11118.287795,80525.058030,32996.785018,19447.298319,107885.760000,52634.300075,33289.151541,44594.545626,32807.068966,15504.352393,32613.028016
3,2018-04-01,건입동,11928.152409,103666.390994,32784.623009,18544.462774,56713.466667,50423.624625,38375.738513,46064.750219,38741.415385,15263.421472,33976.176171
4,2018-05-01,건입동,12378.112014,74175.581937,33580.338901,20405.593880,84271.500000,49263.059602,36232.578479,35229.278632,38258.810526,16030.831987,33646.099706
5,2018-06-01,건입동,11755.872279,84397.037604,34419.728013,21403.454910,73274.314286,45691.888259,35625.395690,32201.013559,42385.200000,14762.096330,32605.412329
6,2018-07-01,건입동,11951.132341,84718.605900,33621.443862,20913.829449,85762.890909,41593.997744,34099.053972,33165.981818,36593.475862,15428.404022,31511.447429
7,2018-08-01,건입동,11599.600820,69973.292025,35389.029312,21412.439147,188771.929412,47260.372046,37999.364376,32053.021243,32582.828571,15278.100077,31491.271825
8,2018-09-01,건입동,12764.825998,112558.801948,33616.246524,22811.906810,76765.618182,52754.537378,35899.521522,31450.946062,36956.183206,15436.116012,34715.714286
9,2018-10-01,건입동,11583.336214,89800.987531,34732.723350,22839.251734,54136.145455,50710.018210,32200.311425,39086.426041,38848.676056,14967.731596,32318.013443


In [334]:
card_df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1848 entries, 0 to 1847
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Year_Month  1848 non-null   datetime64[ns]
 1   emd_nm      1848 non-null   object        
 2   간식          1848 non-null   float64       
 3   농축수산물       1848 non-null   float64       
 4   마트/슈퍼마켓     1848 non-null   float64       
 5   배달          1847 non-null   float64       
 6   부페          1535 non-null   float64       
 7   식품          1848 non-null   float64       
 8   아시아음식       1848 non-null   float64       
 9   양식          1844 non-null   float64       
 10  주점및주류판매     1843 non-null   float64       
 11  패스트푸드       1848 non-null   float64       
 12  한식          1848 non-null   float64       
dtypes: datetime64[ns](1), float64(11), object(1)
memory usage: 187.8+ KB


In [335]:
# pkl 확장자로 저장하면 데이터타입이 안 날라간대요

card_df_new.to_pickle("card_df.pkl")

In [336]:
card = pd.read_pickle("card_df.pkl")

In [337]:
card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1848 entries, 0 to 1847
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Year_Month  1848 non-null   datetime64[ns]
 1   emd_nm      1848 non-null   object        
 2   간식          1848 non-null   float64       
 3   농축수산물       1848 non-null   float64       
 4   마트/슈퍼마켓     1848 non-null   float64       
 5   배달          1847 non-null   float64       
 6   부페          1535 non-null   float64       
 7   식품          1848 non-null   float64       
 8   아시아음식       1848 non-null   float64       
 9   양식          1844 non-null   float64       
 10  주점및주류판매     1843 non-null   float64       
 11  패스트푸드       1848 non-null   float64       
 12  한식          1848 non-null   float64       
dtypes: datetime64[ns](1), float64(11), object(1)
memory usage: 187.8+ KB
